In [1]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from scipy.fft import fft, fftshift, fftfreq    # FFT and helper functions
import numpy as np


from scipy.signal import hilbert, resample_poly
import librosa

input_file = "/Users/christofferaaseth/Documents/GitHub/hydrophonic-detection/Sound_data/001029.wav"

#Getting data from wav file
data_offcet, Fs = librosa.load(input_file)



#Removing Dc-offcet from data
dc_offcet = np.mean(data_offcet)
data = data_offcet - dc_offcet

t = np.linspace(0,(len(data_offcet)/Fs),len(data_offcet))

#Hilbert transform ov data
analytic_signal = hilbert(data)
#Squaring each element
h_2 = np.square(np.absolute(analytic_signal))

#Moving avg filter
med_filt_size = 1001
h_filt = signal.medfilt(h_2,med_filt_size)

#Downsampling
DS_h = resample_poly(h_filt,1,med_filt_size)
DS_Fs = Fs/med_filt_size
DS_t = np.linspace(0,(len(DS_h)/DS_Fs),len(DS_h))




#plotting results
plt.figure(figsize=(12,8))

plt.subplot(3,2,1)
plt.plot(t,data_offcet)
plt.title("Original signal")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

plt.subplot(3,2,2)
plt.plot(t,data)
plt.title("Removing DC-offcet")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

plt.subplot(3,2,3)
plt.plot(t,analytic_signal)
plt.title("Hilbert transform of signal")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

plt.subplot(3,2,4)
plt.plot(t,h_2)
plt.title("Squared Hilbert transform")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

plt.subplot(3,2,5)
plt.plot(t,h_filt)
plt.title("Med_filt result")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

plt.subplot(3,2,6)
plt.plot(DS_t,DS_h)
plt.title("Downsampled med_filt result")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")


NameError: name 'data_off' is not defined